In [1]:
import numpy as np
from numpy.linalg import LinAlgError
import scipy
from datetime import datetime
from collections import defaultdict


from scipy.optimize.linesearch import scalar_search_wolfe2

class LineSearchTool(object):
    """
    Line search tool for adaptively tuning the step size of the algorithm.

    method : String containing 'Wolfe', 'Armijo' or 'Constant'
        Method of tuning step-size.
        Must be be one of the following strings:
            - 'Wolfe' -- enforce strong Wolfe conditions;
            - 'Armijo" -- adaptive Armijo rule;
            - 'Constant' -- constant step size.
    kwargs :
        Additional parameters of line_search method:

        If method == 'Wolfe':
            c1, c2 : Constants for strong Wolfe conditions
            alpha_0 : Starting point for the backtracking procedure
                to be used in Armijo method in case of failure of Wolfe method.
        If method == 'Armijo':
            c1 : Constant for Armijo rule
            alpha_0 : Starting point for the backtracking procedure.
        If method == 'Constant':
            c : The step size which is returned on every step.
    """
    def __init__(self, method='Wolfe', **kwargs):
        self._method = method
        if self._method == 'Wolfe':
            self.c1 = kwargs.get('c1', 1e-4)
            self.c2 = kwargs.get('c2', 0.9)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Armijo':
            self.c1 = kwargs.get('c1', 1e-4)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Constant':
            self.c = kwargs.get('c', 1.0)
        else:
            raise ValueError('Unknown method {}'.format(method))

    @classmethod
    def from_dict(cls, options):
        if type(options) != dict:
            raise TypeError('LineSearchTool initializer must be of type dict')
        return cls(**options)

    def to_dict(self):
        return self.__dict__

    def line_search(self, oracle, x_k, d_k, previous_alpha=None):
        """
        Finds the step size alpha for a given starting point x_k
        and for a given search direction d_k that satisfies necessary
        conditions for func_phi(alpha) = oracle.func(x_k + alpha * d_k).

        Parameters
        ----------
        oracle : BaseSmoothOracle-descendant object
            Oracle with .func_directional() and .grad_directional() methods implemented for computing
            function values and its directional derivatives.
        x_k : np.array
            Starting point
        d_k : np.array
            Search direction
        previous_alpha : float or None
            Starting point to use instead of self.alpha_0 to keep the progress from
             previous steps. If None, self.alpha_0, is used as a starting point.

        Returns
        -------
        alpha : float or None if failure
            Chosen step size
        """
        constant = lambda: self.c
        
        def armijo():
            alpha = previous_alpha or self.alpha_0
            while func_phi(alpha) > (phi0 + alpha * dphi0 * self.c1):
                alpha /= 2
            return alpha
        
        def wolfe():
            alpha = scalar_search_wolfe2(func_phi, func_phi_der, phi0, None, dphi0, self.c1, self.c2)[0]
            if alpha:
                return alpha
            return armijo()
        
        func_phi = lambda x: oracle.func_directional(x_k, d_k, x)
        func_phi_der = lambda x: oracle.grad_directional(x_k, d_k, x)
        phi0 = func_phi(0)
        dphi0 = func_phi_der(0)
        
        methods_dict = {
            'Wolfe': wolfe,
            'Armijo': armijo,
            'Constant': constant
        }

        return methods_dict[self._method]()

def get_line_search_tool(line_search_options=None):
    if line_search_options:
        if type(line_search_options) is LineSearchTool:
            return line_search_options
        else:
            return LineSearchTool.from_dict(line_search_options)
    else:
        return LineSearchTool()


def gradient_descent(oracle, x_0, tolerance=1e-5, max_iter=10000,
                     line_search_options=None, trace=False, display=False):
    """
    Gradien descent optimization method.

    Parameters
    ----------
    oracle : BaseSmoothOracle-descendant object
        Oracle with .func(), .grad() and .hess() methods implemented for computing
        function value, its gradient and Hessian respectively.
    x_0 : np.array
        Starting point for optimization algorithm
    tolerance : float
        Epsilon value for stopping criterion.
    max_iter : int
        Maximum number of iterations.
    line_search_options : dict, LineSearchTool or None
        Dictionary with line search options. See LineSearchTool class for details.
    trace : bool
        If True, the progress information is appended into history dictionary during training.
        Otherwise None is returned instead of history.
    display : bool
        If True, debug information is displayed during optimization.
        Printing format and is up to a student and is not checked in any way.

    Returns
    -------
    x_star : np.array
        The point found by the optimization procedure
    message : string
        "success" or the description of error:
            - 'iterations_exceeded': if after max_iter iterations of the method x_k still doesn't satisfy
                the stopping criterion.
            - 'computational_error': in case of getting Infinity or None value during the computations.
    history : dictionary of lists or None
        Dictionary containing the progress information or None if trace=False.
        Dictionary has to be organized as follows:
            - history['time'] : list of floats, containing time in seconds passed from the start of the method
            - history['func'] : list of function values f(x_k) on every step of the algorithm
            - history['grad_norm'] : list of values Euclidian norms ||g(x_k)|| of the gradient on every step of the algorithm
            - history['x'] : list of np.arrays, containing the trajectory of the algorithm. ONLY STORE IF x.size <= 2

    Example:
    --------
    >> oracle = QuadraticOracle(np.eye(5), np.arange(5))
    >> x_opt, message, history = gradient_descent(oracle, np.zeros(5), line_search_options={'method': 'Armijo', 'c1': 1e-4})
    >> print('Found optimal point: {}'.format(x_opt))
       Found optimal point: [ 0.  1.  2.  3.  4.]
    """
    history = defaultdict(list) if trace else None
    line_search_tool = get_line_search_tool(line_search_options)
    x_k = np.copy(x_0)
    
    alpha_k = None
    stop = tolerance * (np.linalg.norm(oracle.grad(x_0)) ** 2)
    start_time = datetime.now()

    for it in range(max_iter + 1):
        grad_k = oracle.grad(x_k)
        alpha_k = line_search_tool.line_search(oracle, x_k, -grad_k, 2 * alpha_k if alpha_k else None)
        check = display_and_check(
            it, x_k, np.linalg.norm(grad_k), stop, oracle, history, display, trace, start_time)
        if check:
            return check
        x_k = x_k - grad_k * alpha_k
    return x_k, 'iterations_exceeded', history


def newton(oracle, x_0, tolerance=1e-5, max_iter=100,
           line_search_options=None, trace=False, display=False):
    """
    Newton's optimization method.

    Parameters
    ----------
    oracle : BaseSmoothOracle-descendant object
        Oracle with .func(), .grad() and .hess() methods implemented for computing
        function value, its gradient and Hessian respectively. If the Hessian
        returned by the oracle is not positive-definite method stops with message="newton_direction_error"
    x_0 : np.array
        Starting point for optimization algorithm
    tolerance : float
        Epsilon value for stopping criterion.
    max_iter : int
        Maximum number of iterations.
    line_search_options : dict, LineSearchTool or None
        Dictionary with line search options. See LineSearchTool class for details.
    trace : bool
        If True, the progress information is appended into history dictionary during training.
        Otherwise None is returned instead of history.
    display : bool
        If True, debug information is displayed during optimization.

    Returns
    -------
    x_star : np.array
        The point found by the optimization procedure
    message : string
        'success' or the description of error:
            - 'iterations_exceeded': if after max_iter iterations of the method x_k still doesn't satisfy
                the stopping criterion.
            - 'newton_direction_error': in case of failure of solving linear system with Hessian matrix (e.g. non-invertible matrix).
            - 'computational_error': in case of getting Infinity or None value during the computations.
    history : dictionary of lists or None
        Dictionary containing the progress information or None if trace=False.
        Dictionary has to be organized as follows:
            - history['time'] : list of floats, containing time passed from the start of the method
            - history['func'] : list of function values f(x_k) on every step of the algorithm
            - history['grad_norm'] : list of values Euclidian norms ||g(x_k)|| of the gradient on every step of the algorithm
            - history['x'] : list of np.arrays, containing the trajectory of the algorithm. ONLY STORE IF x.size <= 2

    Example:
    --------
    >> oracle = QuadraticOracle(np.eye(5), np.arange(5))
    >> x_opt, message, history = newton(oracle, np.zeros(5), line_search_options={'method': 'Constant', 'c': 1.0})
    >> print('Found optimal point: {}'.format(x_opt))
       Found optimal point: [ 0.  1.  2.  3.  4.]
    """
    history = defaultdict(list) if trace else None
    line_search_tool = get_line_search_tool(line_search_options)
    x_k, norm0 = np.copy(x_0), np.linalg.norm(oracle.grad(x_0)) ** 2
    stop = tolerance * norm0
    start = datetime.now()

    for it in range(max_iter + 1):
        grad_k = oracle.grad(x_k)
        check = display_and_check(it, x_k, np.linalg.norm(grad_k), stop, oracle, history, display, trace, start)
        if check:
            return check
        try:
            hess = oracle.hess(x_k)
            d_k = scipy.linalg.cho_solve(scipy.linalg.cho_factor(hess), -grad_k)
        except LinAlgError:
            return x_k, 'computational_error', history
        a_k = line_search_tool.line_search(oracle, x_k, d_k)
        x_k = x_k + d_k * a_k
    
    return x_k, 'iterations_exceeded', history

def displaying(iteration, x, norm):
    print("iteration {0:4}:".format(iteration))
    print("x = {0}".format(x))
    print("norm = {0}".format(norm))

def fit_history(history, x, start, func, norm):
    if x.size <= 2:
        history['x'].append(x)
    history['time'].append(datetime.now() - start)
    history['func'].append(func)
    history['grad_norm'].append(norm)

def display_and_check(it, x, norm, stop, oracle, history, display, trace, start):
    if display:
        displaying(it, x, norm)
    if trace:
        fit_history(history, x, start, oracle.func(x), norm)
    if norm ** 2 <= stop:
        return x, 'success', history

In [2]:
import numpy as np
import scipy
from scipy.special import expit


class BaseSmoothOracle(object):
    """
    Base class for implementation of oracles.
    """
    def func(self, x):
        """
        Computes the value of function at point x.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, x):
        """
        Computes the gradient at point x.
        """
        raise NotImplementedError('Grad oracle is not implemented.')
    
    def hess(self, x):
        """
        Computes the Hessian matrix at point x.
        """
        raise NotImplementedError('Hessian oracle is not implemented.')
    
    def func_directional(self, x, d, alpha):
        """
        Computes phi(alpha) = f(x + alpha*d).
        """
        return np.squeeze(self.func(x + alpha * d))

    def grad_directional(self, x, d, alpha):
        """
        Computes phi'(alpha) = (f(x + alpha*d))'_{alpha}
        """
        return np.squeeze(self.grad(x + alpha * d).dot(d))


class QuadraticOracle(BaseSmoothOracle):
    """
    Oracle for quadratic function:
       func(x) = 1/2 x^TAx - b^Tx.
    """

    def __init__(self, A, b):
        if not scipy.sparse.isspmatrix_dia(A) and not np.allclose(A, A.T):
            raise ValueError('A should be a symmetric matrix.')
        self.A = A
        self.b = b

    def func(self, x):
        return 0.5 * np.dot(self.A.dot(x), x) - self.b.dot(x)

    def grad(self, x):
        return self.A.dot(x) - self.b

    def hess(self, x):
        return self.A 


class LogRegL2Oracle(BaseSmoothOracle):
    """
    Oracle for logistic regression with l2 regularization:
         func(x) = 1/m sum_i log(1 + exp(-b_i * a_i^T x)) + regcoef / 2 ||x||_2^2.

    Let A and b be parameters of the logistic regression (feature matrix
    and labels vector respectively).
    For user-friendly interface use create_log_reg_oracle()

    Parameters
    ----------
        matvec_Ax : function
            Computes matrix-vector product Ax, where x is a vector of size n.
        matvec_ATx : function of x
            Computes matrix-vector product A^Tx, where x is a vector of size m.
        matmat_ATsA : function
            Computes matrix-matrix-matrix product A^T * Diag(s) * A,
    """
    def __init__(self, matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef):
        self.matvec_Ax = matvec_Ax
        self.matvec_ATx = matvec_ATx
        self.matmat_ATsA = matmat_ATsA
        self.b = b
        self.regcoef = regcoef

    def func(self, x):
        sum_log = np.logaddexp(0, - self.b * self.matvec_Ax(x))
        return np.linalg.norm(sum_log, 1) / self.b.size + np.linalg.norm(x, 2) ** 2 * self.regcoef / 2

    def grad(self, x):
        return self.regcoef * x - self.matvec_ATx(self.b * (expit(-self.b * self.matvec_Ax(x)))) / self.b.size
    
    def hess(self, x):
        tmp = expit(self.b * self.matvec_Ax(x))
        return self.matmat_ATsA(tmp * (1 - tmp)) / self.b.size + self.regcoef * np.identity(x.size)

class LogRegL2OptimizedOracle(LogRegL2Oracle):
    """
    Oracle for logistic regression with l2 regularization
    with optimized *_directional methods (are used in line_search).
    For explanation see LogRegL2Oracle.
    """

    def __init__(self, matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef):
        super().__init__(matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef)
        self.prev_x = None
        self.prev_Ax = None
        self.prev_new_x = None
        self.prev_d = None
        self.prev_Ad = None
        self.prev_new_Ax = None

    def check_Av(self, v, key=''):
        x_to_res = dict((('prev_x', 'prev_Ax'), ('prev_d', 'prev_Ad'), ('prev_new_x', 'prev_new_Ax')))
        for u, k in x_to_res.items():
            tmp = getattr(self, u, None)
            if tmp is not None and np.all(tmp == v):
                return getattr(self, k)

        if key:
            Av = self.matvec_Ax(v)
            setattr(self, x_to_res[key], Av)
            setattr(self, key, v)
            return Av

    def func(self, x):
        Ax = self.check_Av(x, 'prev_x')
        return np.linalg.norm(np.logaddexp(0, -self.b * Ax), 1) / self.b.size + \
               (self.regcoef * np.linalg.norm(x, 2) ** 2) / 2

    def grad(self, x):
        Ax = self.check_Av(x, 'prev_x')
        return self.regcoef * x - self.matvec_ATx(self.b * (expit(- self.b * Ax))) / self.b.size

    def hess(self, x):
        Ax = self.check_Av(x, 'prev_x')
        tmp = expit(self.b * Ax)
        return self.matmat_ATsA(tmp * (1 - tmp)) / self.b.size + self.regcoef * np.identity(x.size)

    def func_directional(self, x, d, alpha):
        new_x = x + alpha * d

        new_Ax = self.check_Av(new_x)
        if new_Ax is None:
            Ax = self.check_Av(x, 'prev_x')
            Ad = self.check_Av(d, 'prev_d')
            new_Ax = Ax + alpha * Ad
            self.prev_new_x = new_x
            self.prev_new_Ax = new_Ax

        return np.linalg.norm(np.logaddexp(0, -self.b * new_Ax), 1) / self.b.size + \
               (self.regcoef * np.linalg.norm(new_x, 2) ** 2) / 2

    def grad_directional(self, x, d, alpha):
        new_x = x + alpha * d

        new_Ax = self.check_Av(new_x)
        Ad = self.check_Av(d, 'prev_d')
        if new_Ax is None:
            Ax = self.check_Av(x, 'prev_x')
            new_Ax = Ax + alpha * Ad
            self.prev_new_x = new_x
            self.prev_new_Ax = new_Ax

        return self.regcoef * np.dot(np.transpose(new_x), d) - \
               np.dot(self.b * (expit(- self.b * new_Ax)), Ad) / self.b.size


def create_log_reg_oracle(A, b, regcoef, oracle_type='usual'):
    """
    Auxiliary function for creating logistic regression oracles.
        `oracle_type` must be either 'usual' or 'optimized'
    """
    matvec_Ax = lambda x: A.dot(x)
    matvec_ATx = lambda x: A.T.dot(x)

    if scipy.sparse.issparse(A):
        A = scipy.sparse.csr_matrix(A)
        matvec_Ax = lambda x: A.dot(x)
        matvec_ATx = lambda x: A.T.dot(x)
        matmat_ATsA = lambda x: matvec_ATx(matvec_ATx(scipy.sparse.diags(x)).T)
    else:
        matmat_ATsA = lambda x: np.dot(matvec_ATx(np.diag(x)), A)
    
    if oracle_type == 'usual':
        oracle = LogRegL2Oracle
    elif oracle_type == 'optimized':
        oracle = LogRegL2OptimizedOracle
    else:
        raise 'Unknown oracle type=%s' % oracle_type
    return oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef)


def grad_finite_diff(func, x, eps=1e-8):
    """
    Returns approximation of the gradient using finite differences:
        result_i := (f(x + eps * e_i) - f(x)) / eps,
        where e_i are coordinate vectors:
        e_i = (0, 0, ..., 0, 1, 0, ..., 0)
                          >> i <<
    """
    x = np.array(x)
    f_x = func(x)
    e = np.identity(x.size)
    res = np.zeros_like(x)

    for i in range(x.size):
        res[i] = (func(x + e[i] * eps) - f_x) / eps
    
    return res


def hess_finite_diff(func, x, eps=1e-5):
    """
    Returns approximation of the Hessian using finite differences:
        result_{ij} := (f(x + eps * e_i + eps * e_j)
                               - f(x + eps * e_i)
                               - f(x + eps * e_j)
                               + f(x)) / eps^2,
        where e_i are coordinate vectors:
        e_i = (0, 0, ..., 0, 1, 0, ..., 0)
                          >> i <<
    """
    x = np.array(x)
    f_x = func(x)
    res, tmp, e = np.zeros((x.size, x.size)), np.zeros_like(x), np.identity(x.size)

    for i in range(x.size):
        tmp[i] = func(x + e[i] * eps)

    for i in range(x.size):
        for j in range(x.size):
            res[i][j] = (func(x + eps * (e[i] + e[j]))- tmp[j] - tmp[i] + f_x) / (eps ** 2)

    return res

In [3]:
import nose
from nose.tools import assert_almost_equal, ok_, eq_
from nose.plugins.attrib import attr
from io import StringIO
import numpy as np
import scipy
import scipy.sparse
import scipy.optimize
import sys
import warnings

def test_python3():
    ok_(sys.version_info > (3, 0))


def test_QuadraticOracle():
    # Quadratic function:
    #   f(x) = 1/2 x^T x - [1, 2, 3]^T x
    A = np.eye(3)
    b = np.array([1, 2, 3])
    quadratic = QuadraticOracle(A, b)

    # Check at point x = [0, 0, 0]
    x = np.zeros(3)
    assert_almost_equal(quadratic.func(x), 0.0)
    ok_(np.allclose(quadratic.grad(x), -b))
    ok_(np.allclose(quadratic.hess(x), A))
    ok_(isinstance(quadratic.grad(x), np.ndarray))
    ok_(isinstance(quadratic.hess(x), np.ndarray))

    # Check at point x = [1, 1, 1]
    x = np.ones(3)
    assert_almost_equal(quadratic.func(x), -4.5)
    ok_(np.allclose(quadratic.grad(x), x - b))
    ok_(np.allclose(quadratic.hess(x), A))
    ok_(isinstance(quadratic.grad(x), np.ndarray))
    ok_(isinstance(quadratic.hess(x), np.ndarray))

    # Check func_direction and grad_direction oracles at
    # x = [1, 1, 1], d = [-1, -1, -1], alpha = 0.5 and 1.0
    x = np.ones(3)
    d = -np.ones(3)
    assert_almost_equal(quadratic.func_directional(x, d, alpha=0.5),
                        -2.625)
    assert_almost_equal(quadratic.grad_directional(x, d, alpha=0.5),
                        4.5)
    assert_almost_equal(quadratic.func_directional(x, d, alpha=1.0),
                        0.0)
    assert_almost_equal(quadratic.grad_directional(x, d, alpha=1.0),
                        6.0)


def check_log_reg(oracle_type, sparse=False):
    # Simple data:
    A = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    if sparse: A = scipy.sparse.csr_matrix(A)
    b = np.array([1, 1, -1, 1])
    reg_coef = 0.5

    # Logistic regression oracle:
    logreg = create_log_reg_oracle(A, b, reg_coef, oracle_type=oracle_type)

    # Check at point x = [0, 0]
    x = np.zeros(2)
    assert_almost_equal(logreg.func(x), 0.693147180)
    ok_(np.allclose(logreg.grad(x), [0, -0.25]))
    ok_(np.allclose(logreg.hess(x), [[0.625, 0.0625], [0.0625, 0.625]]))
    ok_(isinstance(logreg.grad(x), np.ndarray))
    ok_(isinstance(logreg.hess(x), np.ndarray))

    # Check func_direction and grad_direction oracles at
    # x = [0, 0], d = [1, 1], alpha = 0.5 and 1.0
    x = np.zeros(2)
    d = np.ones(2)
    assert_almost_equal(logreg.func_directional(x, d, alpha=0.5),
                        0.7386407091095)
    assert_almost_equal(logreg.grad_directional(x, d, alpha=0.5),
                        0.4267589549159)
    assert_almost_equal(logreg.func_directional(x, d, alpha=1.0),
                        1.1116496416598)
    assert_almost_equal(logreg.grad_directional(x, d, alpha=1.0),
                        1.0559278283039)


def test_log_reg_usual():
    check_log_reg('usual')
    check_log_reg('usual', sparse=True)


@attr('bonus')
def test_log_reg_optimized():
    check_log_reg('optimized')
    check_log_reg('optimized', sparse=True)


def get_counters(A):
    counters = {'Ax': 0, 'ATx': 0, 'ATsA': 0}

    def matvec_Ax(x):
        counters['Ax'] += 1
        return A.dot(x)

    def matvec_ATx(x):
        counters['ATx'] += 1
        return A.T.dot(x)

    def matmat_ATsA(s):
        counters['ATsA'] += 1
        return A.T.dot(A * s.reshape(-1, 1))

    return (matvec_Ax, matvec_ATx, matmat_ATsA, counters)


def check_counters(counters, groundtruth):
    for (key, value) in groundtruth.items():
        ok_(key in counters)
        ok_(counters[key] <= value)


def test_log_reg_oracle_calls():

    A = np.ones((2, 2))
    b = np.ones(2)
    x = np.ones(2)
    d = np.ones(2)
    reg_coef = 0.5

    # Single func
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).func(x)
    check_counters(counters, {'Ax': 1, 'ATx': 0, 'ATsA': 0})

    # Single grad
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).grad(x)
    check_counters(counters, {'Ax': 1, 'ATx': 1, 'ATsA': 0})

    # Single hess
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).hess(x)
    check_counters(counters, {'Ax': 1, 'ATx': 0, 'ATsA': 1})

    # Single func_directional
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).func_directional(x, d, 1)
    check_counters(counters, {'Ax': 1, 'ATx': 0, 'ATsA': 0})

    # Single grad_directional
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).grad_directional(x, d, 1)
    check_counters(counters, {'Ax': 1, 'ATx': 1, 'ATsA': 0})

    # In a row: func + grad + hess
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    oracle.hess(x)
    check_counters(counters, {'Ax': 3, 'ATx': 1, 'ATsA': 1})

    # In a row: func + grad
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    check_counters(counters, {'Ax': 2, 'ATx': 1, 'ATsA': 0})

    # In a row: grad + hess
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.grad(x)
    oracle.hess(x)
    check_counters(counters, {'Ax': 2, 'ATx': 1, 'ATsA': 1})

    # In a row: func + grad + func_directional + grad_directional
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    oracle.func_directional(x, d, 1)
    oracle.grad_directional(x, d, 2)
    oracle.func_directional(x, d, 2)
    oracle.func_directional(x, d, 3)
    check_counters(counters, {'Ax': 6, 'ATx': 2, 'ATsA': 0})

    # In a row: func + grad + func_directional + grad_directional + (func + grad)
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2Oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    oracle.func_directional(x, d, 1)
    oracle.grad_directional(x, d, 2)
    oracle.func_directional(x, d, 2)
    oracle.func_directional(x, d, 3)
    oracle.func(x + 3 * d)
    oracle.grad(x + 3 * d)
    check_counters(counters, {'Ax': 8, 'ATx': 3, 'ATsA': 0})


@attr('bonus')
def test_log_reg_optimized_oracle_calls():

    A = np.ones((2, 2))
    b = np.ones(2)
    x = np.ones(2)
    d = np.ones(2)
    reg_coef = 0.5

    # Single func
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).func(x)
    check_counters(counters, {'Ax': 1, 'ATx': 0, 'ATsA': 0})

    # Single grad
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).grad(x)
    check_counters(counters, {'Ax': 1, 'ATx': 1, 'ATsA': 0})

    # Single hess
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).hess(x)
    check_counters(counters, {'Ax': 1, 'ATx': 0, 'ATsA': 1})

    # Single func_directional
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).func_directional(x, d, 1)
    check_counters(counters, {'Ax': 2, 'ATx': 0, 'ATsA': 0})

    # Single grad_directional
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef).grad_directional(x, d, 1)
    check_counters(counters, {'Ax': 2, 'ATx': 0, 'ATsA': 0})

    # In a row: func + grad + hess
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    oracle.hess(x)
    check_counters(counters, {'Ax': 1, 'ATx': 1, 'ATsA': 1})

    # In a row: func + grad
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    check_counters(counters, {'Ax': 1, 'ATx': 1, 'ATsA': 0})

    # In a row: grad + hess
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.grad(x)
    oracle.hess(x)
    check_counters(counters, {'Ax': 1, 'ATx': 1, 'ATsA': 1})

    # In a row: func + grad + func_directional + grad_directional
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    oracle.func_directional(x, d, 1)
    oracle.grad_directional(x, d, 2)
    oracle.func_directional(x, d, 2)
    oracle.func_directional(x, d, 3)
    check_counters(counters, {'Ax': 2, 'ATx': 1, 'ATsA': 0})

    # In a row: func + grad + func_directional + grad_directional + (func + grad)
    (matvec_Ax, matvec_ATx, matmat_ATsA, counters) = get_counters(A)
    oracle = LogRegL2OptimizedOracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, reg_coef)
    oracle.func(x)
    oracle.grad(x)
    oracle.func_directional(x, d, 1)
    oracle.grad_directional(x, d, 2)
    oracle.func_directional(x, d, 2)
    oracle.func_directional(x, d, 3)
    oracle.func(x + 3 * d)
    oracle.grad(x + 3 * d)
    check_counters(counters, {'Ax': 2, 'ATx': 2, 'ATsA': 0})


def test_grad_finite_diff_1():
    # Quadratic function.
    A = np.eye(3)
    b = np.array([1, 2, 3])
    quadratic = QuadraticOracle(A, b)
    g = grad_finite_diff(quadratic.func, np.zeros(3))
    ok_(isinstance(g, np.ndarray))
    ok_(np.allclose(g, -b))


def test_grad_finite_diff_2():
    # f(x, y) = x^3 + y^2
    func = lambda x: x[0] ** 3 + x[1] ** 2
    x = np.array([2.0, 3.0])
    eps = 1e-5
    g = grad_finite_diff(func, x, eps)
    ok_(isinstance(g, np.ndarray))
    ok_(np.allclose(g, [12.0, 6.0], atol=1e-4))


def test_hess_finite_diff_1():
    # Quadratic function.
    A = np.eye(3)
    b = np.array([1, 2, 3])
    quadratic = QuadraticOracle(A, b)
    H = hess_finite_diff(quadratic.func, np.zeros(3))
    ok_(isinstance(H, np.ndarray))
    ok_(np.allclose(H, A))


def test_hess_finite_diff_2():
    # f(x, y) = x^3 + y^2
    func = lambda x: x[0] ** 3 + x[1] ** 2
    x = np.array([2.0, 3.0])
    eps = 1e-5
    H = hess_finite_diff(func, x, eps)
    ok_(isinstance(H, np.ndarray))
    ok_(np.allclose(H, [[12.0, 0.], [0., 2.0]], atol=1e-3))


def get_quadratic():
    # Quadratic function:
    #   f(x) = 1/2 x^T x - [1, 2, 3]^T x
    A = np.eye(3)
    b = np.array([1, 2, 3])
    return QuadraticOracle(A, b)


def test_line_search():
    oracle = get_quadratic()
    x = np.array([100, 0, 0])
    d = np.array([-1, 0, 0])

    # Constant line search
    ls_tool = LineSearchTool(method='Constant', c=1.0)
    assert_almost_equal(ls_tool.line_search(oracle, x, d, ), 1.0)
    ls_tool = LineSearchTool(method='Constant', c=10.0)
    assert_almost_equal(ls_tool.line_search(oracle, x, d), 10.0)

    # Armijo rule
    ls_tool = LineSearchTool(method='Armijo', alpha_0=100, c1=0.9)
    assert_almost_equal(ls_tool.line_search(oracle, x, d), 12.5)

    ls_tool = LineSearchTool(method='Armijo', alpha_0=100, c1=0.9)
    assert_almost_equal(ls_tool.line_search(oracle, x, d, previous_alpha=1.0), 1.0)

    ls_tool = LineSearchTool(method='Armijo', alpha_0=100, c1=0.95)
    assert_almost_equal(ls_tool.line_search(oracle, x, d), 6.25)
    ls_tool = LineSearchTool(method='Armijo', alpha_0=10, c1=0.9)
    assert_almost_equal(ls_tool.line_search(oracle, x, d), 10.0)

    # Wolfe rule
    ls_tool = LineSearchTool(method='Wolfe', c1=1e-4, c2=0.9)
    assert_almost_equal(ls_tool.line_search(oracle, x, d), 16.0)
    ls_tool = LineSearchTool(method='Wolfe', c1=1e-4, c2=0.8)
    assert_almost_equal(ls_tool.line_search(oracle, x, d), 32.0)


def check_equal_histories(history1, history2, atol=1e-3):
    if history1 is None or history2 is None:
        eq_(history1, history2)
        return
    ok_('func' in history1 and 'func' in history2)
    ok_(np.allclose(history1['func'], history2['func'], atol=atol))
    ok_('grad_norm' in history1 and 'grad_norm' in history2)
    ok_(np.allclose(history1['grad_norm'], history2['grad_norm'], atol=atol))
    ok_('time' in history1 and 'time' in history2)
    eq_(len(history1['time']), len(history2['time']))
    eq_('x' in history1, 'x' in history2)
    if 'x' in history1:
        ok_(np.allclose(history1['x'], history2['x'], atol=atol))


def check_prototype(method):
    class ZeroOracle2D(BaseSmoothOracle):
        def func(self, x): return 0.0

        def grad(self, x): return np.zeros(2)

        def hess(self, x): return np.zeros([2, 2])

    oracle = ZeroOracle2D()
    x0 = np.ones(2)
    HISTORY = {'func': [0.0],
               'grad_norm': [0.0],
               'time': [0],  # dummy timestamp
               'x': [np.ones(2)]}

    def check_result(result, x0=np.ones(2), msg='success', history=None):
        eq_(len(result), 3)
        ok_(np.allclose(result[0], x0))
        eq_(result[1], msg)
        check_equal_histories(result[2], history)

    check_result(method(oracle, x0))
    check_result(method(oracle, x0, 1e-3, 10))
    check_result(method(oracle, x0, 1e-3, 10, {'method': 'Constant', 'c': 1.0}))
    check_result(method(oracle, x0, 1e-3, 10, {'method': 'Constant', 'c': 1.0},
                        trace=True), history=HISTORY)
    check_result(method(oracle, x0, 1e-3, max_iter=10,
                        line_search_options={'method': 'Constant', 'c': 1.0},
                        trace=True, display=True), history=HISTORY)
    check_result(method(oracle, x0, display=True, trace=False))
    check_result(method(oracle, x0, tolerance=1e-8, trace=True),
                 history=HISTORY)

    # Check default display=False
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    check_result(method(oracle, x0))
    eq_(mystdout.getvalue(), "")
    sys.stdout = old_stdout
    # Check specified display=False
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    check_result(method(oracle, x0, display=False))
    eq_(mystdout.getvalue(), "")
    sys.stdout = old_stdout
    # Check specified display=True
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    check_result(method(oracle, x0, display=True))
    ok_(len(mystdout.getvalue()) > 1)
    sys.stdout = old_stdout


def check_one_ideal_step(method):
    oracle = get_quadratic()
    x0 = np.ones(3) * 10.0
    [x_star, msg, history] = method(oracle, x0, max_iter=1,
                                    tolerance=1e-5, trace=True)
    ok_(np.allclose(x_star, [1.0, 2.0, 3.0]))
    eq_(msg, 'success')
    check_equal_histories(history, {'func': [90.0, -7.0],
                                    'grad_norm': [13.928388277184119, 0.0],
                                    'time': [0, 1]  # dummy timestamps
                                    })


def test_gd_basic():
    check_prototype(gradient_descent)
    check_one_ideal_step(gradient_descent)


def test_newton_basic():
    check_prototype(newton)
    check_one_ideal_step(newton)


def get_1d(alpha):
    # 1D function:
    #   f(x) = exp(alpha * x) + alpha * x^2
    class Func(BaseSmoothOracle):
        def __init__(self, alpha):
            self.alpha = alpha

        def func(self, x):
            return np.exp(self.alpha * x) + self.alpha * x ** 2

        def grad(self, x):
            return np.array(self.alpha * np.exp(self.alpha * x) +
                            2 * self.alpha * x)

        def hess(self, x):
            return np.array([self.alpha ** 2 * np.exp(self.alpha * x) +
                             2 * self.alpha])

    return Func(alpha)


def test_gd_1d():
    oracle = get_1d(0.5)
    x0 = np.array([1.0])
    FUNC = [
        np.array([2.14872127]),
        np.array([0.8988787]),
        np.array([0.89869501]),
        np.array([0.89869434]),
        np.array([0.89869434])]
    GRAD_NORM = [
        1.8243606353500641,
        0.021058536428132546,
        0.0012677045924299746,
        7.5436847232768223e-05,
        4.485842052370792e-06]
    TIME = [0] * 5  # Dummy values.
    X = [
        np.array([1.]),
        np.array([-0.42528175]),
        np.array([-0.40882976]),
        np.array([-0.40783937]),
        np.array([-0.40778044])]
    TRUE_HISTORY = {'func': FUNC,
                    'grad_norm': GRAD_NORM,
                    'time': TIME,
                    'x': X}
    # Armijo rule.
    [x_star, msg, history] = gradient_descent(
        oracle, x0,
        max_iter=5,
        tolerance=1e-10,
        trace=True,
        line_search_options={
            'method': 'Armijo',
            'alpha_0': 100,
            'c1': 0.3
        }
    )
    ok_(np.allclose(x_star, [-0.4077], atol=1e-3))
    eq_(msg, 'success')
    check_equal_histories(history, TRUE_HISTORY)
    # Constant step size.
    [x_star, msg, history] = gradient_descent(oracle, x0,
                                                           max_iter=5, tolerance=1e-10, trace=False,
                                                           line_search_options={
                                                               'method': 'Constant',
                                                               'c': 1.0})
    ok_(np.allclose(x_star, [-0.4084371], atol=1e-2))
    eq_(msg, 'iterations_exceeded')
    eq_(history, None)


def test_newton_1d():
    oracle = get_1d(0.5)
    x0 = np.array([1.0])
    FUNC = [
        np.array([2.14872127]),
        np.array([0.9068072]),
        np.array([0.89869455]),
        np.array([0.89869434])]
    GRAD_NORM = [
        1.8243606353500641,
        0.14023069594489929,
        0.00070465169721295462,
        1.7464279966628027e-08]
    TIME = [0] * 4  # Dummy values.
    X = [
        np.array([1.]),
        np.array([-0.29187513]),
        np.array([-0.40719141]),
        np.array([-0.40777669])]
    TRUE_HISTORY = {'func': FUNC,
                    'grad_norm': GRAD_NORM,
                    'time': TIME,
                    'x': X}
    # Constant step size.
    [x_star, msg, history] = newton(
        oracle, x0,
        max_iter=5, tolerance=1e-10, trace=True,
        line_search_options={
            'method': 'Constant',
            'c': 1.0}
    )
    ok_(np.allclose(x_star, [-0.4077777], atol=1e-4))
    eq_(msg, 'success')
    check_equal_histories(history, TRUE_HISTORY)


def test_newton_fail():
    # f(x) = integral_{-infty}^x arctan(t) dt
    class Oracle(BaseSmoothOracle):
        def func(self, x):
            return x * np.arctan(x) - 0.5 * np.log(np.power(x, 2) + 1)

        def grad(self, x):
            return np.arctan(x)

        def hess(self, x):
            return np.array([1 / (np.power(x, 2) + 1)])

    x0 = np.array([10.0])
    warnings.filterwarnings("ignore")
    [x_star, msg, history] = newton(Oracle(), x0,
                                                display=False, trace=False,
                                                 line_search_options={'method': 'Constant', 'c': 1})
    warnings.filterwarnings("default")
    eq_(msg, 'computational_error')
    eq_(history, None)

In [4]:
test_python3()

In [5]:
test_QuadraticOracle()

In [6]:
test_log_reg_usual()

In [7]:
test_log_reg_optimized()

In [8]:
test_log_reg_oracle_calls()

In [9]:
test_log_reg_optimized_oracle_calls()

In [10]:
test_grad_finite_diff_1()

In [11]:
test_grad_finite_diff_2()

In [12]:
test_hess_finite_diff_1()

In [13]:
test_hess_finite_diff_2()

In [14]:
test_line_search()

In [15]:
test_gd_basic()

iteration    0:
x = [1. 1.]
norm = 0.0
iteration    0:
x = [1. 1.]
norm = 0.0


In [16]:
test_newton_basic()

iteration    0:
x = [1. 1.]
norm = 0.0
iteration    0:
x = [1. 1.]
norm = 0.0


In [17]:
test_gd_1d()

In [18]:
test_newton_1d()

In [19]:
test_newton_fail()